In [1]:
import time
import os
import numpy as np
import pandas as pd
import pytz

nrows=100000
#nrows=None

input_dir = '/home/kai/data/kaggle/talkingdata/data/'
work_dir  = '../work'
dtype_train = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'is_attributed' : 'uint8',
        'click_time': object,
        }
dtype_test = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'click_id'      : 'uint32',
        'click_time': object,
        }

train_df = pd.read_csv(input_dir+"/train.csv", dtype=dtype_train, usecols=dtype_train.keys(), nrows=nrows, parse_dates=['click_time'])
test_df = pd.read_csv(input_dir+"/test_supplement.csv", dtype=dtype_test, usecols=dtype_test.keys(), nrows=nrows, parse_dates=['click_time'])
train_df['click_id'] = 0


In [4]:
# need to check
ans_file = "/home/kai/data/kaggle/talkingdata/data/ans_full.csv"
if os.path.exists(ans_file):
    df_ans_full = pd.read_csv(ans_file,nrows=nrows)
    test_df['is_attributed'] = df_ans_full['is_attributed']
else:
    test_df['is_attributed'] = 0
len_train = len(train_df)
df = train_df.append(test_df).reset_index()
cst = pytz.timezone('Asia/Shanghai')
df['datetime'] = pd.to_datetime(df['click_time']).dt.tz_localize(pytz.utc).dt.tz_convert(cst)
df['day'] = df.datetime.dt.day.astype('uint8')
df['click_time'] = (df['click_time'].astype(np.int64) // 10 ** 9).astype(np.int32)
df = df.sort_values(['click_time','is_attributed','click_id'])[['click_time','day','ip','app','device','os']]

In [24]:
df.head()

,click_time,day,ip,app,device,os
0,1509978741,6,83230,3,1,13
1,1509978814,6,17357,3,1,19
2,1509978852,6,35810,3,1,13
3,1509978892,6,45745,14,1,13
4,1509978908,6,161007,3,1,13


In [3]:
name = 'nextClickLeakDay'
# the time elapsed to the combination appears next time
# e.g. if one combination appears at time 14, and it appears next at time 15, then we will assign 15 - 14 = 1 to 
# the row of time 14.
# if it never appears again, then assgin 999999 to it
df[name] = (df.groupby(['day', 'ip', 'app', 'device', 'os']).click_time.shift(-1) - df.click_time+1).fillna(999999).astype(int)
out_df = df[[name]].sort_index()
out_df[len_train:].to_csv(work_dir + '/test_supplement_' + name + '.csv', index=False)
out_df[:len_train].to_csv(work_dir + '/train_' + name + '.csv', index=False)
print(work_dir + '/test_supplement_' + name + '.csv')
print(work_dir + '/train_' + name + '.csv')

name = 'nextNextClickLeakDay'
# the time elapsed to the combination appears next next time
# e.g. if one combination appears at time 14, and it appears next next at time 20, then we will assign 20 - 14 = 6 to 
# the row of time 14.
df[name] = (df.groupby(['day', 'ip', 'app', 'device', 'os']).click_time.shift(-2) - df.click_time+1).fillna(999999).astype(int)
out_df = df[[name]].sort_index()
out_df[len_train:].to_csv(work_dir + '/test_supplement_' + name + '.csv', index=False)
out_df[:len_train].to_csv(work_dir + '/train_' + name + '.csv', index=False)
print(work_dir + '/test_supplement_' + name + '.csv')
print(work_dir + '/train_' + name + '.csv')


../work/test_supplement_nextClickLeakDay.csv
../work/train_nextClickLeakDay.csv
../work/test_supplement_nextNextClickLeakDay.csv
../work/train_nextNextClickLeakDay.csv


In [5]:
temp = df.sample(frac=0.01)

In [7]:
ts = (temp.groupby(['day', 'ip', 'app', 'device', 'os']).click_time.shift(-1) - temp.click_time).reset_index()

In [8]:
ts.shape

(2000, 2)

In [9]:
ts[ts.click_time.notnull()] # note the negative value could appear because after sampling, the order is changed

,index,click_time
46,30630,65.0
64,12589,70.0
66,66717,-68.0
222,79862,-13.0
375,16911,-3.0
382,158013,14.0
390,85734,4.0
498,182438,-15.0
500,94388,2.0
550,44611,-16.0


In [10]:
tl = ts[ts.click_time.notnull()]['index'].tolist()

In [11]:
tl

[30630,
 12589,
 66717,
 79862,
 16911,
 158013,
 85734,
 182438,
 94388,
 44611,
 198542,
 171390,
 186157,
 154575,
 86374,
 62976,
 122448,
 176861]

In [12]:
shifted = temp.loc[tl]
shifted

,click_time,day,ip,app,device,os
30630,1509984037,7,73487,15,1,19
12589,1509984017,7,2160,18,1,19
66717,1509984071,7,14901,20,1,17
79862,1509984085,7,153411,9,1,98
16911,1509984022,7,206746,18,1,28
158013,1510243255,10,119349,7,1,18
85734,1509984092,7,22211,3,1,19
182438,1510243276,10,48240,9,1,17
94388,1509984102,7,73487,15,1,19
44611,1509984049,7,100867,8,1,14


In [13]:
cols = ['day', 'ip', 'app', 'device', 'os']

In [14]:
temp.reset_index(inplace=True)

In [17]:
showmewhy = shifted.merge(temp, how='left', on=cols, suffixes=('_after', '_before'))
showmewhy

,click_time_after,day,ip,app,device,os,index,click_time_before
0,1509984037,7,73487,15,1,19,30630,1509984037
1,1509984037,7,73487,15,1,19,94388,1509984102
2,1509984037,7,73487,15,1,19,96577,1509984104
3,1509984017,7,2160,18,1,19,12589,1509984017
4,1509984017,7,2160,18,1,19,81186,1509984087
5,1509984071,7,14901,20,1,17,66717,1509984071
6,1509984071,7,14901,20,1,17,2329,1509984003
7,1509984085,7,153411,9,1,98,79862,1509984085
8,1509984085,7,153411,9,1,98,67643,1509984072
9,1509984022,7,206746,18,1,28,16911,1509984022


In [20]:
train_df.head()

,ip,app,device,os,click_time,is_attributed,click_id
0,83230,3,1,13,2017-11-06 14:32:21,0,0
1,17357,3,1,19,2017-11-06 14:33:34,0,0
2,35810,3,1,13,2017-11-06 14:34:12,0,0
3,45745,14,1,13,2017-11-06 14:34:52,0,0
4,161007,3,1,13,2017-11-06 14:35:08,0,0


# let's see how utc and general time string are related precisely

In [21]:
index_and_clicktime = train_df.reset_index()[['index','click_time']]

In [22]:
index_and_clicktime.head()

,index,click_time
0,0,2017-11-06 14:32:21
1,1,2017-11-06 14:33:34
2,2,2017-11-06 14:34:12
3,3,2017-11-06 14:34:52
4,4,2017-11-06 14:35:08


In [23]:
showmewhy.merge(index_and_clicktime, how='left', on='index')

,click_time_after,day,ip,app,device,os,index,click_time_before,click_time
0,1509984037,7,73487,15,1,19,30630,1509984037,2017-11-06 16:00:37
1,1509984037,7,73487,15,1,19,94388,1509984102,2017-11-06 16:01:42
2,1509984037,7,73487,15,1,19,96577,1509984104,2017-11-06 16:01:44
3,1509984017,7,2160,18,1,19,12589,1509984017,2017-11-06 16:00:17
4,1509984017,7,2160,18,1,19,81186,1509984087,2017-11-06 16:01:27
5,1509984071,7,14901,20,1,17,66717,1509984071,2017-11-06 16:01:11
6,1509984071,7,14901,20,1,17,2329,1509984003,2017-11-06 16:00:03
7,1509984085,7,153411,9,1,98,79862,1509984085,2017-11-06 16:01:25
8,1509984085,7,153411,9,1,98,67643,1509984072,2017-11-06 16:01:12
9,1509984022,7,206746,18,1,28,16911,1509984022,2017-11-06 16:00:22
